# Tracking Bugs

_Brief abstract/introduction/motivation.  State what the chapter is about in 1-2 paragraphs._
_Then, have an introduction video:_

In [ ]:
from bookutils import YouTubeVideo
# YouTubeVideo("w4u5gCgPlmg")

**Prerequisites**

* _Refer to earlier chapters as notebooks here, as here:_ [Earlier Chapter](Debugger.ipynb)

In [ ]:
import bookutils

In [ ]:
import Intro_Debugging

In [ ]:
import os

In [ ]:
# ignore
#
# WARNING: Unlike the other chapters in this book, 
# this chapter should NOT BE RUN AS A NOTEBOOK:
#
# * It will delete ALL data from an existing 
#   local _RedMine_ installation.
# * It will create new users and databases in an existing
#   local _MySQL_ installation.
# 
# The only reason to run this notebook is to create the book chapter,
# which is the task of Andreas Zeller.
# If you are not Andreas, you should exactly know what you are doing.

assert os.getenv('USER') == 'zeller'

## A Change Tracker

We have a look at the _Redmine_ change tracking system.

### Excursion: Setting up Redmine

To install Redmine, we followed the instructions at https://gist.github.com/johnjohndoe/2763243.
These final steps initialize the database:

In [ ]:
import subprocess

In [ ]:
import os
import sys

In [ ]:
def with_ruby(cmd, inp='', timeout=10, show_stdout=False):
    print(f"$ {cmd}")
    shell = subprocess.Popen(['/bin/sh', '-c',
        f'''rvm_redmine=$HOME/.rvm/gems/ruby-2.7.2@redmine; \
rvm_global=$HOME/.rvm/gems/ruby-2.7.2@global; \
export GEM_PATH=$rvm_redmine:$rvm_global; \
export PATH=$rvm_redmine/bin:$rvm_global/bin:$HOME/.rvm/rubies/ruby-2.7.2/bin:$HOME/.rvm/bin:$PATH; \
cd $HOME/lib/redmine && {cmd}'''],
                             stdin=subprocess.PIPE,
                             stdout=subprocess.PIPE,
                             stderr=subprocess.PIPE,
                             universal_newlines=True)
    try:
        stdout_data, stderr_data = shell.communicate(inp, timeout=timeout)
    except subprocess.TimeoutExpired:
        shell.kill()
#         stdout_data, stderr_data = shell.communicate(inp)
#         if show_stdout:
#             print(stdout_data, end="")
#         print(stderr_data, file=sys.stderr, end="")
        raise

    print(stderr_data, file=sys.stderr, end="")
    if show_stdout:
        print(stdout_data, end="")

In [ ]:
def with_mysql(cmd, timeout=2, show_stdout=False):
    print(f"sql>{cmd}")
    sql = subprocess.Popen(["mysql", "-u", "root",
                           "--default-character-set=utf8mb4"],
                            stdin=subprocess.PIPE,
                            stdout=subprocess.PIPE,
                            stderr=subprocess.PIPE, 
                            universal_newlines=True)
    try:
        stdout_data, stderr_data = sql.communicate(cmd + ';', 
                                                   timeout=timeout)
    except suprocess.TimeoutExpired:
        sql.kill()
#         stdout_data, stderr_data = sql.communicate(inp)
#         if show_stdout:
#             print(stdout_data, end="")
#         print(stderr_data, file=sys.stderr, end="")
        raise

    print(stderr_data, file=sys.stderr, end="")
    if show_stdout:
        print(stdout_data, end="")

In [ ]:
with_ruby("bundle config set without development test")

In [ ]:
with_ruby("bundle install")

In [ ]:
with_ruby("pkill sql; sleep 5")

In [ ]:
try:
    with_ruby("mysql.server start", show_stdout=True)
except subprocess.TimeoutExpired:
    pass  # Can actually start without producing output

In [ ]:
with_mysql("drop database redmine")

In [ ]:
with_mysql("drop user 'redmine'@'localhost'")

In [ ]:
with_mysql("create database redmine character set utf8")

In [ ]:
with_mysql("create user 'redmine'@'localhost' identified by 'my_password'")

In [ ]:
with_mysql("grant all privileges on redmine.* to 'redmine'@'localhost'")

In [ ]:
with_ruby("bundle exec rake generate_secret_token")

In [ ]:
with_ruby("RAILS_ENV=production bundle exec rake db:migrate")

In [ ]:
with_ruby("RAILS_ENV=production bundle exec rake redmine:load_default_data", '\n')

### End of Excursion

### Excursion: Starting Redmine

In [ ]:
import os
import time

In [ ]:
from multiprocessing import Process

In [ ]:
def run_redmine(port):
    with_ruby(f'exec rails s -e production -p {port} > redmine.log 2>&1',
             timeout=600)

In [ ]:
def start_redmine(port=3000):
    process = Process(target=run_redmine, args=(port,))
    process.start()
    time.sleep(5)

    url = f"http://localhost:{port}"
    return process, url

In [ ]:
redmine_process, redmine_url = start_redmine()

### End of Excursion

### Excursion: Remote Control with Selenium

We assume is that there is a set of *user interface elements* we can interact with.

[Selenium](https://www.seleniumhq.org) is a framework for testing Web applications by _automating interaction in the browser_.  Selenium provides an API that allows one to launch a Web browser, query the state of the user interface, and interact with individual user interface elements.  The Selenium API is available in a number of languages; we use the [Selenium API for Python](https://selenium-python.readthedocs.io/index.html).

A Selenium *web driver* is the interface between a program and a browser controlled by the program.

In [ ]:
from selenium import webdriver

In [ ]:
from selenium.webdriver.common.keys import Keys

The following code starts a Firefox browser in the background, which we then control through the web driver.

In [ ]:
BROWSER = 'firefox'

In [ ]:
with_ruby("pkill Firefox.app firefox-bin")

**Note:** If you don't have Firefox installed, you can also set `BROWSER` to `'chrome'` to use Google Chrome instead.

In [ ]:
# BROWSER = 'chrome'

When running this outside of Jupyter notebooks, the browser is _headless_, meaning that it does not show on the screen.

In [ ]:
from bookutils import rich_output

In [ ]:
# HEADLESS = not rich_output()
HEADLESS = False

In [ ]:
def start_webdriver(browser=BROWSER, headless=HEADLESS, zoom=1.4):
    if browser == 'firefox':
        options = webdriver.FirefoxOptions()
    if browser == 'chrome':
        options = webdriver.ChromeOptions()

    if headless and browser == 'chrome':
        options.add_argument('headless')
    else:
        options.headless = headless

    # Start the browser, and obtain a _web driver_ object such that we can interact with it.
    if browser == 'firefox':
        # For firefox, set a higher resolution for our screenshots
        profile = webdriver.firefox.firefox_profile.FirefoxProfile()
        profile.set_preference("layout.css.devPixelsPerPx", repr(zoom))
        gui_driver = webdriver.Firefox(firefox_profile=profile, options=options)

        # We set the window size such that it fits
        gui_driver.set_window_size(500, 600)  # was 1024, 600

    elif browser == 'chrome':
        gui_driver = webdriver.Chrome(options=options)
        gui_driver.set_window_size(1024, 510 if headless else 640)

    return gui_driver

In [ ]:
gui_driver = start_webdriver(browser=BROWSER, headless=HEADLESS)

We can now interact with the browser programmatically.  First, we have it navigate to the URL of our Web server:

In [ ]:
gui_driver.get(redmine_url)

To see what the "headless" browser displays, we can obtain a screenshot.  We see that it actually displays the home page.

In [ ]:
from IPython.display import display, Image

In [ ]:
Image(gui_driver.get_screenshot_as_png())

### End of Excursion

### Excursion: Screenshots with Drop Shadows

By default, our screenshots are flat. We add a drop shadow to make them look nicer.
With help from https://graphicdesign.stackexchange.com/questions/117272/how-to-add-drop-shadow-to-a-picture-via-cli

In [ ]:
import tempfile

In [ ]:
def drop_shadow(contents):
    with tempfile.NamedTemporaryFile() as tmp:
        tmp.write(contents)
        convert = subprocess.Popen(
            ['convert', tmp.name,
            '(', '+clone', '-background', 'black', '-shadow', '50x10+15+15', ')',
            '+swap', '-background', 'none', '-layers', 'merge', '+repage', '-'],
            stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        stdout_data, stderr_data = convert.communicate()
    
    if stderr_data:
        print(stderr_data.decode("utf-8"), file=sys.stderr, end="")
        
    return stdout_data

In [ ]:
def screenshot(driver):
    return Image(drop_shadow(gui_driver.get_screenshot_as_png()))

In [ ]:
screenshot(gui_driver)

### End of Excursion

### Excursion: First Registration at Redmine

In [ ]:
gui_driver.get(redmine_url + '/login')

In [ ]:
screenshot(gui_driver)

In [ ]:
gui_driver.find_element_by_id("username").send_keys("admin")
gui_driver.find_element_by_id("password").send_keys("admin")
gui_driver.find_element_by_name("login").click()

In [ ]:
time.sleep(2)

In [ ]:
if gui_driver.current_url.endswith('my/password'):
    gui_driver.get(redmine_url + '/my/password')
    gui_driver.find_element_by_id("password").send_keys("admin")
    gui_driver.find_element_by_id("new_password").send_keys("admin001")
    gui_driver.find_element_by_id("new_password_confirmation").send_keys("admin001")
    display(screenshot(gui_driver))
    gui_driver.find_element_by_name("commit").click()

In [ ]:
gui_driver.get(redmine_url + '/logout')
gui_driver.find_element_by_name("commit").click()

### End of Excursion

This is what the _Redmine_ tracker starts with:

In [ ]:
# ignore
gui_driver.get(redmine_url + '/login')
screenshot(gui_driver)

After we login, we see our account:

In [ ]:
# ignore
gui_driver.find_element_by_id("username").send_keys("admin")
gui_driver.find_element_by_id("password").send_keys("admin001")
gui_driver.find_element_by_name("login").click()
screenshot(gui_driver)

### Excursion: Creating a Project

Let us check out the projects by clicking on "Projects".

In [ ]:
# ignore
gui_driver.get(redmine_url + '/projects')
screenshot(gui_driver)

Let us create a new project by clicking on "Projects".

In [ ]:
# ignore
gui_driver.get(redmine_url + '/projects/new')
screenshot(gui_driver)

In [ ]:
# ignore
gui_driver.get(redmine_url + '/projects/new')
gui_driver.find_element_by_id('project_name').send_keys("The Debugging Book")
gui_driver.find_element_by_id('project_description').send_keys("A Book on Automated Debugging")
gui_driver.find_element_by_id('project_identifier').clear()
gui_driver.find_element_by_id('project_identifier').send_keys("debuggingbook")
gui_driver.find_element_by_id('project_homepage').send_keys("https://www.debuggingbook.org/")
screenshot(gui_driver)

In [ ]:
# ignore
gui_driver.find_element_by_name('commit').click()

### End of Excursion

We start with a list of projects.

In [ ]:
# ignore
gui_driver.get(redmine_url + '/projects')
screenshot(gui_driver)

Let us choose the (one) "debuggingbook" project.

In [ ]:
# ignore
gui_driver.get(redmine_url + '/projects/debuggingbook')
screenshot(gui_driver)

## Reporting a Bug

Let us file a new issue:

In [ ]:
# ignore
gui_driver.get(redmine_url + '/issues/new')
screenshot(gui_driver)

Let's give our bug a name:

In [ ]:
issue_title = "Does not render correctly on Nokia Communicator"

In [ ]:
issue_description = \
"""The Debugging Book does not render correctly on the Nokia Communicator 9000.

Steps to reproduce:
1. On the Nokia, go to "https://debuggingbook.org/"
2. From the menu on top, select the chapter "Tracking Origins".
3. Scroll down to a place where a graph is supposed to be shown.
4. Instead of the graph, only a blank space is displayed.

How to fix:
* The graphs seem to come as SVG elements, but the Nokia Communicator
does not support SVG rendering. Render them as JPEGs instead.
"""

In [ ]:
# ignore
gui_driver.get(redmine_url + '/issues/new')

gui_driver.find_element_by_id('issue_subject').send_keys(issue_title)
gui_driver.find_element_by_id('issue_description').send_keys(issue_description)
screenshot(gui_driver)

In [ ]:
# ignore
gui_driver.find_element_by_id('issue_assigned_to_id').click()
screenshot(gui_driver)

In [ ]:
# ignore
gui_driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
screenshot(gui_driver)

In [ ]:
# ignore
gui_driver.find_element_by_name('commit').click()
screenshot(gui_driver)

## What Goes in a Bug Report?

## The Life Cycle of a Bug

We have successfully filed a bug. Now, this bug will enter the life cycle.

See also https://www.softwaretestinghelp.com/bug-life-cycle/

In [ ]:
from Intro_Debugging import graph

In [ ]:
from IPython.display import display

In [ ]:
# ignore
life_cycle = graph()
life_cycle.attr(rankdir='TB')

life_cycle.node('New', label="<<b>NEW</b>>", penwidth='2.0')
life_cycle.node('Assigned', label="<<b>ASSIGNED</b>>")

with life_cycle.subgraph() as res:
    res.attr(rank='same')
    res.node('Resolved', label="<<b>RESOLVED</b>>", penwidth='2.0')
    res.node('Resolution',
                shape='plain',
                fillcolor='white',
                label="""<<b>Resolution:</b> One of<br align="left"/>
• FIXED<br align="left"/>
• DUPLICATE<br align="left"/>
• WONTFIX<br align="left"/>
• WORKSFORME<br align="left"/>
• INVALID<br align="left"/>
>""")
    res.node('Reopened', label="<<b>REOPENED</b>>", style='invis')

life_cycle.edge('New', 'Assigned', label=r"Assigned\lto developer")
life_cycle.edge('Assigned', 'Resolved', label="Developer has fixed bug")

life_cycle.edge('Resolution', 'Resolved', arrowhead='none', style='dashed')

life_cycle

In [ ]:
# ignore

life_cycle.node('Unconfirmed', label="<<b>UNCONFIRMED</b>>", penwidth='2.0')
life_cycle.node('Verified', label="<<b>VERIFIED</b>>")
life_cycle.node('Closed', label="<<b>CLOSED</b>>", penwidth='2.0')
life_cycle.node('Reopened', label="<<b>REOPENED</b>>", style='filled')
life_cycle.node('New', label="<<b>NEW</b>>", penwidth='1.0')

life_cycle.edge('Unconfirmed', 'New', label="Confirmed as \"new\"")
life_cycle.edge('Assigned', 'New', label="Re-assigned")
life_cycle.edge('Resolved', 'Verified', label=r"Quality Assurance\lconfirms fix")
life_cycle.edge('Resolved', 'Reopened', label=r"Quality Assurance\lnot satisfied")
life_cycle.edge('Reopened', 'Assigned', label=r"Assigned\lto developer")
life_cycle.edge('Verified', 'Closed', label="Bug is closed")
life_cycle.edge('Closed', 'Reopened', label=r"Bug is\lreopened")

life_cycle

### Assigning Bug Reports

## Prioritizing Bug Reports

To experiment with RedMine and other issue trackers yourself, pick one of the test installations:

* \todo{add}

## Cleanup

In [ ]:
import os

In [ ]:
redmine_process.terminate()

In [ ]:
gui_driver.close()

In [ ]:
os.system("pkill ruby")

## Synopsis

<!-- Automatically generated. Do not edit. -->



_For those only interested in using the code in this chapter (without wanting to know how it works), give an example.  This will be copied to the beginning of the chapter (before the first section) as text with rendered input and output._

You can use `int_fuzzer()` as:

```python
print(int_fuzzer())
```
```python
=> 76.5

```


## Synopsis

_For those only interested in using the code in this chapter (without wanting to know how it works), give an example.  This will be copied to the beginning of the chapter (before the first section) as text with rendered input and output._

You can use `int_fuzzer()` as:

## Lessons Learned

* _Lesson one_
* _Lesson two_
* _Lesson three_

## Next Steps

_Link to subsequent chapters (notebooks) here, as in:_


## Background

_Cite relevant works in the literature and put them into context, as in:_

The idea of ensuring that each expansion in the grammar is used at least once goes back to Burkhardt \cite{Burkhardt1967}, to be later rediscovered by Paul Purdom \cite{Purdom1972}.

## Exercises

_Close the chapter with a few exercises such that people have things to do.  To make the solutions hidden (to be revealed by the user), have them start with_

```
**Solution.**
```

_Your solution can then extend up to the next title (i.e., any markdown cell starting with `#`)._

_Running `make metadata` will automatically add metadata to the cells such that the cells will be hidden by default, and can be uncovered by the user.  The button will be introduced above the solution._

### Exercise 1: _Title_

_Text of the exercise_

In [ ]:
# Some code that is part of the exercise
pass

_Some more text for the exercise_

**Solution.** _Some text for the solution_

In [ ]:
# Some code for the solution
2 + 2

_Some more text for the solution_

### Exercise 2: _Title_

_Text of the exercise_

**Solution.** _Solution for the exercise_